https://www.kaggle.com/tejasrinivas/xgb-baseline-comments-classification
    
    3598 views
    32 voters

In [1]:
#The features are borrowed from Eike Dehling's kernel at  https://www.kaggle.com/eikedehling/feature-engineering


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

train = train.fillna("unknown")
test = test.fillna("unknown")


train_mes, valid_mes, train_l, valid_l = train_test_split(train['comment_text'],train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']], test_size=0.2, random_state=2)



#Using the tokenize function from Jeremy's kernel
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

transform_com = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(train['comment_text'])
'''comments_train = transform_com.transform(train['comment_text'])'''
comments_train = transform_com.transform(train_mes)
comments_valid = transform_com.transform(valid_mes)
comments_test = transform_com.transform(test['comment_text'])
gc.collect()

train_mes = pd.DataFrame(train_mes)
valid_mes = pd.DataFrame(valid_mes)

data = [train_mes, valid_mes, test]

for element in data:
    element['total_length'] = element['comment_text'].apply(len)
    element['capitals'] = element['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    element['caps_vs_length'] = element.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
    element['num_exclamation_marks'] = element['comment_text'].apply(lambda comment: comment.count('!'))
    element['num_question_marks'] = element['comment_text'].apply(lambda comment: comment.count('?'))
    element['num_punctuation'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
    element['num_symbols'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
    element['num_words'] = element['comment_text'].apply(lambda comment: len(comment.split()))
    element['num_unique_words'] = element['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
    element['words_vs_unique'] = element['num_unique_words'] / element['num_words']
    element['num_smilies'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
    
col = ['total_length', 'capitals', 'caps_vs_length',
       'num_exclamation_marks', 'num_question_marks', 'num_punctuation',
       'num_symbols', 'num_words', 'num_unique_words', 'words_vs_unique',
       'num_smilies']

train_mes = scipy.sparse.csr_matrix(train_mes[col].values)
valid_mes = scipy.sparse.csr_matrix(valid_mes[col].values)
test = scipy.sparse.csr_matrix(test[col].values)


comments_train = scipy.sparse.hstack([train_mes.tocsr(),comments_train.tocsr()])
comments_valid = scipy.sparse.hstack([valid_mes,comments_valid])
comments_test = scipy.sparse.hstack([test,comments_test])


import xgboost as xgb


def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2017, num_rounds=500):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'auc'
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    return model    

col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((test.shape[0], len(col)))

for i, j in enumerate(col):
    print('fit '+j)
    model = runXGB(comments_train, train_l[j], comments_valid,valid_l[j])
    preds[:,i] = model.predict(xgb.DMatrix(comments_test), ntree_limit = model.best_ntree_limit)
    gc.collect()

subm = pd.read_csv('../input/sample_submission.csv')    
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = col)], axis=1)
submission.to_csv('submission_xgb_baseline.csv', index=False)
    

# Any results you write to the current directory are saved as output.

crawl-300d-2M.vec
glove.6B.50d.txt
glove.840B.300d.txt
randomstate
sample_submission.csv
test.csv
train.csv
wordbatch

fit toxic
[0]	train-auc:0.699241	test-auc:0.698428
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 20 rounds.
[1]	train-auc:0.750327	test-auc:0.747547
[2]	train-auc:0.771126	test-auc:0.766802
[3]	train-auc:0.815439	test-auc:0.81078
[4]	train-auc:0.845785	test-auc:0.84086
[5]	train-auc:0.848276	test-auc:0.843024
[6]	train-auc:0.850965	test-auc:0.845957
[7]	train-auc:0.855177	test-auc:0.85012
[8]	train-auc:0.866715	test-auc:0.860291
[9]	train-auc:0.866933	test-auc:0.859988
[10]	train-auc:0.876527	test-auc:0.870098
[11]	train-auc:0.879838	test-auc:0.87224
[12]	train-auc:0.888496	test-auc:0.8804
[13]	train-auc:0.890955	test-auc:0.882645
[14]	train-auc:0.895053	test-auc:0.887679
[15]	train-auc:0.901173	test-auc:0.893962
[16]	train-auc:0.903185	test-auc:0.895958
[17]	train-auc:0.904841	test-auc

[189]	train-auc:0.9781	test-auc:0.959573
[190]	train-auc:0.978199	test-auc:0.95956
[191]	train-auc:0.978277	test-auc:0.959604
[192]	train-auc:0.978399	test-auc:0.959655
[193]	train-auc:0.978438	test-auc:0.959657
[194]	train-auc:0.978477	test-auc:0.95971
[195]	train-auc:0.978569	test-auc:0.959756
[196]	train-auc:0.978634	test-auc:0.959868
[197]	train-auc:0.978757	test-auc:0.959921
[198]	train-auc:0.978803	test-auc:0.960019
[199]	train-auc:0.978899	test-auc:0.960093
[200]	train-auc:0.978941	test-auc:0.960158
[201]	train-auc:0.979014	test-auc:0.96026
[202]	train-auc:0.979103	test-auc:0.960336
[203]	train-auc:0.979198	test-auc:0.960505
[204]	train-auc:0.979333	test-auc:0.96051
[205]	train-auc:0.979325	test-auc:0.960468
[206]	train-auc:0.979451	test-auc:0.960446
[207]	train-auc:0.979527	test-auc:0.960487
[208]	train-auc:0.979567	test-auc:0.960507
[209]	train-auc:0.979652	test-auc:0.960585
[210]	train-auc:0.97978	test-auc:0.960626
[211]	train-auc:0.979952	test-auc:0.960761
[212]	train-auc:0.

[381]	train-auc:0.988243	test-auc:0.967184
[382]	train-auc:0.988264	test-auc:0.967253
[383]	train-auc:0.988283	test-auc:0.967271
[384]	train-auc:0.988321	test-auc:0.967286
[385]	train-auc:0.988353	test-auc:0.967274
[386]	train-auc:0.988385	test-auc:0.967288
[387]	train-auc:0.988413	test-auc:0.967314
[388]	train-auc:0.988431	test-auc:0.967346
[389]	train-auc:0.988466	test-auc:0.967375
[390]	train-auc:0.988541	test-auc:0.967379
[391]	train-auc:0.988587	test-auc:0.967437
[392]	train-auc:0.988617	test-auc:0.967455
[393]	train-auc:0.988648	test-auc:0.967485
[394]	train-auc:0.988678	test-auc:0.967513
[395]	train-auc:0.988692	test-auc:0.967564
[396]	train-auc:0.98871	test-auc:0.967624
[397]	train-auc:0.988762	test-auc:0.967622
[398]	train-auc:0.988807	test-auc:0.967679
[399]	train-auc:0.988856	test-auc:0.967678
[400]	train-auc:0.988883	test-auc:0.9677
[401]	train-auc:0.988908	test-auc:0.967693
[402]	train-auc:0.988945	test-auc:0.967699
[403]	train-auc:0.988982	test-auc:0.967733
[404]	train-au

[71]	train-auc:0.993313	test-auc:0.985292
[72]	train-auc:0.993408	test-auc:0.985388
[73]	train-auc:0.993527	test-auc:0.98526
[74]	train-auc:0.993607	test-auc:0.985147
[75]	train-auc:0.993642	test-auc:0.985202
[76]	train-auc:0.993691	test-auc:0.985321
[77]	train-auc:0.993762	test-auc:0.985316
[78]	train-auc:0.993891	test-auc:0.985699
[79]	train-auc:0.993917	test-auc:0.985986
[80]	train-auc:0.993962	test-auc:0.985995
[81]	train-auc:0.994027	test-auc:0.986006
[82]	train-auc:0.994141	test-auc:0.985996
[83]	train-auc:0.994195	test-auc:0.986114
[84]	train-auc:0.994313	test-auc:0.986206
[85]	train-auc:0.994415	test-auc:0.986212
[86]	train-auc:0.994475	test-auc:0.986225
[87]	train-auc:0.994557	test-auc:0.986247
[88]	train-auc:0.994614	test-auc:0.986412
[89]	train-auc:0.994656	test-auc:0.98644
[90]	train-auc:0.994739	test-auc:0.986452
[91]	train-auc:0.994782	test-auc:0.986513
[92]	train-auc:0.994816	test-auc:0.986497
[93]	train-auc:0.994872	test-auc:0.986539
[94]	train-auc:0.994931	test-auc:0.9

[12]	train-auc:0.893174	test-auc:0.887235
[13]	train-auc:0.905175	test-auc:0.89732
[14]	train-auc:0.90967	test-auc:0.900302
[15]	train-auc:0.91016	test-auc:0.900772
[16]	train-auc:0.915232	test-auc:0.906915
[17]	train-auc:0.916085	test-auc:0.906822
[18]	train-auc:0.935464	test-auc:0.929112
[19]	train-auc:0.935648	test-auc:0.929253
[20]	train-auc:0.938483	test-auc:0.934519
[21]	train-auc:0.938593	test-auc:0.934518
[22]	train-auc:0.938979	test-auc:0.934938
[23]	train-auc:0.944729	test-auc:0.938788
[24]	train-auc:0.945716	test-auc:0.941503
[25]	train-auc:0.950909	test-auc:0.944526
[26]	train-auc:0.951334	test-auc:0.945365
[27]	train-auc:0.952265	test-auc:0.945597
[28]	train-auc:0.956328	test-auc:0.947945
[29]	train-auc:0.956882	test-auc:0.947661
[30]	train-auc:0.957764	test-auc:0.947987
[31]	train-auc:0.963153	test-auc:0.953118
[32]	train-auc:0.964245	test-auc:0.954118
[33]	train-auc:0.96477	test-auc:0.955001
[34]	train-auc:0.965339	test-auc:0.955702
[35]	train-auc:0.967484	test-auc:0.958

...

[398]	train-auc:0.996733	test-auc:0.986275
[399]	train-auc:0.996757	test-auc:0.986267
[400]	train-auc:0.996775	test-auc:0.986291
[401]	train-auc:0.996787	test-auc:0.986272
[402]	train-auc:0.996826	test-auc:0.98627
Stopping. Best iteration:
[382]	train-auc:0.996561	test-auc:0.986313

fit threat
[0]	train-auc:0.706765	test-auc:0.738077
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 20 rounds.
[1]	train-auc:0.733705	test-auc:0.754617
[2]	train-auc:0.751751	test-auc:0.760166
[3]	train-auc:0.759494	test-auc:0.760171
[4]	train-auc:0.769763	test-auc:0.765656
[5]	train-auc:0.781169	test-auc:0.77651
[6]	train-auc:0.791459	test-auc:0.787616
[7]	train-auc:0.791448	test-auc:0.787628
[8]	train-auc:0.797873	test-auc:0.798708
[9]	train-auc:0.803014	test-auc:0.809794
[10]	train-auc:0.811884	test-auc:0.809659
[11]	train-auc:0.824689	test-auc:0.820599
[12]	train-auc:0.825952	test-auc:0.820576
[13]	train-auc:0.847568	test-

[89]	train-auc:0.97806	test-auc:0.965033
[90]	train-auc:0.978225	test-auc:0.965068
[91]	train-auc:0.978505	test-auc:0.965196
[92]	train-auc:0.978667	test-auc:0.965423
[93]	train-auc:0.978734	test-auc:0.965439
[94]	train-auc:0.978965	test-auc:0.965648
[95]	train-auc:0.979106	test-auc:0.965835
[96]	train-auc:0.979171	test-auc:0.965901
[97]	train-auc:0.979377	test-auc:0.966052
[98]	train-auc:0.979546	test-auc:0.966193
[99]	train-auc:0.979742	test-auc:0.966555
[100]	train-auc:0.979783	test-auc:0.966639
[101]	train-auc:0.979866	test-auc:0.966711
[102]	train-auc:0.980014	test-auc:0.966724
[103]	train-auc:0.980114	test-auc:0.966864
[104]	train-auc:0.980271	test-auc:0.967044
[105]	train-auc:0.980421	test-auc:0.967157
[106]	train-auc:0.98057	test-auc:0.967336
[107]	train-auc:0.980724	test-auc:0.967429
[108]	train-auc:0.980797	test-auc:0.967623
[109]	train-auc:0.980854	test-auc:0.967775
[110]	train-auc:0.98094	test-auc:0.967835
[111]	train-auc:0.981061	test-auc:0.967901
[112]	train-auc:0.981182	

[281]	train-auc:0.991527	test-auc:0.976644
[282]	train-auc:0.991545	test-auc:0.976718
[283]	train-auc:0.991568	test-auc:0.976738
[284]	train-auc:0.991594	test-auc:0.976743
[285]	train-auc:0.991636	test-auc:0.976761
[286]	train-auc:0.9917	test-auc:0.976765
[287]	train-auc:0.99172	test-auc:0.976776
[288]	train-auc:0.991731	test-auc:0.976757
[289]	train-auc:0.991751	test-auc:0.976782
[290]	train-auc:0.991796	test-auc:0.976804
[291]	train-auc:0.99181	test-auc:0.976806
[292]	train-auc:0.991821	test-auc:0.976851
[293]	train-auc:0.991894	test-auc:0.976865
[294]	train-auc:0.991904	test-auc:0.976885
[295]	train-auc:0.991915	test-auc:0.976878
[296]	train-auc:0.991941	test-auc:0.976901
[297]	train-auc:0.991975	test-auc:0.976874
[298]	train-auc:0.991997	test-auc:0.976893
[299]	train-auc:0.992016	test-auc:0.976906
[300]	train-auc:0.992054	test-auc:0.977
[301]	train-auc:0.99207	test-auc:0.977075
[302]	train-auc:0.992103	test-auc:0.97708
[303]	train-auc:0.992142	test-auc:0.977085
[304]	train-auc:0.99

fit identity_hate
[0]	train-auc:0.676128	test-auc:0.664651
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 20 rounds.
[1]	train-auc:0.695827	test-auc:0.677656
[2]	train-auc:0.789925	test-auc:0.751763
[3]	train-auc:0.801266	test-auc:0.756806
[4]	train-auc:0.801349	test-auc:0.756944
[5]	train-auc:0.801803	test-auc:0.758944
[6]	train-auc:0.803582	test-auc:0.760697
[7]	train-auc:0.803632	test-auc:0.760769
[8]	train-auc:0.828395	test-auc:0.785181
[9]	train-auc:0.832604	test-auc:0.786764
[10]	train-auc:0.833079	test-auc:0.788635
[11]	train-auc:0.836652	test-auc:0.792497
[12]	train-auc:0.858647	test-auc:0.801306
[13]	train-auc:0.858621	test-auc:0.801304
[14]	train-auc:0.865881	test-auc:0.80856
[15]	train-auc:0.872524	test-auc:0.810291
[16]	train-auc:0.876272	test-auc:0.810388
[17]	train-auc:0.876597	test-auc:0.81024
[18]	train-auc:0.880359	test-auc:0.81385
[19]	train-auc:0.880476	test-auc:0.813858
[20]	train-auc

Stopping. Best iteration:
[170]	train-auc:0.994352	test-auc:0.964098

